In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression

In [4]:
transactions = pd.read_csv('transactions.csv')

In [3]:
#transactions.amount = transactions.amount.apply(lambda x: round(x/11.2296))

In [5]:
train_transactions = transactions[transactions.amount < 0].copy()
train_transactions['day'] = train_transactions.tr_datetime.apply(lambda dt: dt.split()[0]).astype(int)

In [6]:
train_transactions.head()

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,day
0,39026145,0 10:23:26,4814,1030,-2245.92,NaN,0
2,39026145,1 10:20:56,4829,2330,-56147.89,NaN,1
3,39026145,1 10:39:54,5499,1010,-1392.47,NaN,1
4,39026145,2 15:33:42,5499,1010,-920.83,NaN,2
5,39026145,2 15:53:49,5541,1010,-14643.37,NaN,2


In [7]:
test_transactions = pd.DataFrame(columns=train_transactions.mcc_code.unique(), 
                                 index=np.arange(1, 31) + train_transactions.day.max())
test_transactions = test_transactions.unstack().reset_index().dropna(axis=1)
test_transactions.columns = ['mcc_code', 'day']

In [8]:
test_transactions.head()

,mcc_code,day
0,4814,457
1,4814,458
2,4814,459
3,4814,460
4,4814,461


In [9]:
train_grid = pd.DataFrame(columns=train_transactions.mcc_code.unique(), 
                          index=train_transactions.day.unique())
train_grid = train_grid.unstack().reset_index().dropna(axis=1)
train_grid.columns = ['mcc_code', 'day']

In [10]:
train_grid.head()

,mcc_code,day
0,4814,0
1,4814,1
2,4814,2
3,4814,3
4,4814,4


In [11]:
for tr_table in [train_transactions, test_transactions, train_grid]:
    tr_table['week_num'] = tr_table['day'] // 7
    tr_table['week_day'] = tr_table['day'] % 7
    tr_table['month_num'] = tr_table['day'] // 30
    tr_table['month_day'] = tr_table['day'] % 30

In [12]:
train_transactions = \
    pd.merge(train_grid,
             train_transactions.groupby(['day', 'week_num', 'week_day', 'month_num', 'month_day', 'mcc_code'])[['amount']]\
                 .sum().reset_index(),
             how='left').fillna(0)

In [13]:
train_transactions.head()

,mcc_code,day,week_num,week_day,month_num,month_day,amount
0,4814,0,0,0,0,0,-11098744.26
1,4814,1,0,1,0,1,-7881825.53
2,4814,2,0,2,0,2,-6777480.45
3,4814,3,0,3,0,3,-9277943.73
4,4814,4,0,4,0,4,-9999757.21


In [14]:
for day_shift in [-1, 0, 1]:
    for month_shift in train_transactions.month_num.unique()[1:]:
        train_shift = train_transactions.copy()
        train_shift['month_num'] += month_shift
        train_shift['month_day'] += day_shift
        train_shift['amount_day_{}_{}'.format(day_shift, month_shift)] = np.log(-train_shift['amount'] + 1)
        train_shift = train_shift[['month_num', 'month_day', 'mcc_code', 'amount_day_{}_{}'.format(day_shift, month_shift)]]

        train_transactions = pd.merge(train_transactions, train_shift, 
                                      on=['month_num', 'month_day', 'mcc_code'], how='left').fillna(0)
        test_transactions = pd.merge(test_transactions, train_shift, 
                                     on=['month_num', 'month_day', 'mcc_code'], how='left').fillna(0)


In [15]:
train_transactions.head()

,mcc_code,day,week_num,week_day,month_num,month_day,amount,amount_day_-1_1,amount_day_-1_2,amount_day_-1_3,...,amount_day_1_6,amount_day_1_7,amount_day_1_8,amount_day_1_9,amount_day_1_10,amount_day_1_11,amount_day_1_12,amount_day_1_13,amount_day_1_14,amount_day_1_15
0,4814,0,0,0,0,0,-11098744.26,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4814,1,0,1,0,1,-7881825.53,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4814,2,0,2,0,2,-6777480.45,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4814,3,0,3,0,3,-9277943.73,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4814,4,0,4,0,4,-9999757.21,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
day_for_train = train_transactions.day.max() - 30

In [23]:
X_train = train_transactions[train_transactions.day <= day_for_train]
X_val = train_transactions[train_transactions.day > day_for_train]

In [18]:
X_train.day.max(), X_val.day.min()

(426, 427)

In [19]:
shift = 500
train = pd.get_dummies(X_train, columns=['mcc_code'])
val = pd.get_dummies(X_val, columns=['mcc_code'])
test = pd.get_dummies(test_transactions, columns=['mcc_code'])

c = train.columns.difference(['amount'])

In [25]:
clf = LinearRegression(n_jobs=-1)
clf.fit(train[c], np.log(-train['amount'] + shift))

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False)

In [26]:
pred = clf.predict(test[c])

In [27]:
pred.max()

210148419.3081736

In [28]:
np.sum(pred)

-20929744988.460541

In [29]:
def RMSLE(y_true,y_pred):
    return np.sqrt(np.sum(np.square(np.log(y_true + 500) - np.log(y_pred + 500)))/len(pred))

In [68]:
test_transactions['volume'] = np.e ** clf.predict(test[c]) - shift

## xgb

In [18]:
import xgboost as xgb

In [23]:
param = {}
param['max_depth'] = 5
param['booster'] = 'gbtree'
param['objective'] = 'reg:linear'
param['eval_metric'] = 'rmse'
param['eta'] = 0.05
param['subsample'] = 0.7
param['colsample_bytree'] = 0.8
param['colsample_bylevel'] = 0.8

numround = 100

In [31]:
%%time
model = xgb.XGBRegressor(max_depth=20, learning_rate=0.1, n_estimators=200, silent=False, \
                          objective='reg:linear', nthread=24, \
                          subsample=0.6, colsample_bytree=0.6, colsample_bylevel=0.7)
model.fit(train[c], np.log(-train['amount'] + shift))

XGBRegressor(base_score=0.5, colsample_bylevel=0.7, colsample_bytree=0.6,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=20,
       min_child_weight=1, missing=None, n_estimators=200, nthread=24,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=False, subsample=0.6)

In [32]:
test_transactions['volume'] = np.e ** model.predict(test[c]) - shift

In [33]:
test_transactions[['mcc_code', 'day', 'volume']].head()

,mcc_code,day,volume
0,4814,457,9228809.0
1,4814,458,11115197.0
2,4814,459,11007090.0
3,4814,460,9621389.0
4,4814,461,10203106.0


In [34]:
test_transactions[['mcc_code', 'day', 'volume']].to_csv('TASK2_baseline_xgb.csv', index=False)

In [57]:
prev_month = X_val.reset_index()

In [58]:
prev_month = prev_month[["mcc_code", "day", "amount"]]

In [59]:
prev_month.amount = prev_month.amount.apply(lambda x: -x)

In [60]:
prev_month.day = test_transactions.day

In [61]:
prev_month["volume"] = prev_month.amount

In [62]:
prev_month = prev_month.drop(["amount"], axis=1)

In [65]:
prev_month.volume = prev_month.volume.apply(lambda x: abs(x))

In [66]:
prev_month.to_csv('TASK2_prev_month.csv', index=False)

In [70]:
test_transactions[['mcc_code', 'day', 'volume']].head()

,mcc_code,day,volume
0,4814,457,1.205250e+07
1,4814,458,1.227261e+07
2,4814,459,1.236170e+07
3,4814,460,1.225352e+07
4,4814,461,1.251162e+07


In [ ]:
test_transactions[['mcc_code', 'day', 'volume']].to_csv('baseline_b.csv', index=False)